This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/humanoids.html).  I recommend having both windows open, side-by-side!


In [2]:
from functools import partial

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    AddUnitQuaternionConstraintOnPlant,
    AutoDiffXd,
    DiagramBuilder,
    ExtractGradient,
    ExtractValue,
    InitializeAutoDiff,
    JacobianWrtVariable,
    JointIndex,
    MathematicalProgram,
    MeshcatVisualizer,
    MultibodyPlant,
    OrientationConstraint,
    Parser,
    PidController,
    PiecewisePolynomial,
    PositionConstraint,
    RigidTransform,
    RotationMatrix,
    Simulator,
    SnoptSolver,
    Solve,
    StartMeshcat,
    eq,
    namedview,
)

from underactuated import ConfigureParser, running_as_notebook



import math
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pydot
from IPython.display import HTML, display, SVG
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    LeafSystem,
    LinearSystem,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    ModelVisualizer,
    Parser,
    LogVectorOutput,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    SymbolicVectorSystem,
    Variable,
    WrapToSystem,
    FrameId
)
from pydrake.common import FindResourceOrThrow

from pydrake.all import MultibodyPlant, RevoluteJoint, RigidTransform, UnitInertia, RigidBody
from pydrake.multibody.tree import SpatialInertia


from pydrake.examples import (
    AcrobotGeometry,
    AcrobotInput,

    AcrobotPlant,
    AcrobotState,
    QuadrotorGeometry,
    QuadrotorPlant,
    StabilizingLQRController,
)
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.geometry import Cylinder

from underactuated import ConfigureParser, running_as_notebook
from underactuated.meshcat_utils import MeshcatSliders
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer

if running_as_notebook:
    mpld3.enable_notebook()

In [3]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://0561bb78-2b86-4280-965f-ef37e93411a8.deepnoteproject.com/7001/


In [22]:
rod_link = """ 
    <link name="rod0">

        <inertial>
        <origin xyz="0 0 -1/2" />
        <mass value=".225" />
        </inertial>

    <!-- Visual Component -->

        <visual>
        <origin xyz="0 0 0.5" rpy="0 0 0" />
        <geometry>
            <cylinder length="1.0" radius=".01" />
        </geometry>
        <material>
            <color rgba="1 0 0 1" />
        </material>
        </visual>
        <collision>
            <origin xyz="0 0 0.5" rpy="0 0 0" />
            <geometry>
                <cylinder length="1.0" radius=".01" />
            </geometry>
        </collision>

    </link>
    """

base_link = """<link name="base_link">
    <inertial>
      <mass value="0.775"/>
      <origin xyz="0 0 0"/>
      <inertia ixx="0.0015" ixy="0.0" ixz="0.0" iyy="0.0025" iyz="0.0" izz="0.0035"/>
    </inertial>
    <visual>
      <origin rpy="1.570796 0 0" xyz="0 0 0"/>
      <geometry>
        <mesh filename="package://drake_models/skydio_2/skydio_2_1000_poly.obj" scale=".00254"/>
      </geometry>
    </visual>
    <collision>
      <origin rpy="0 0 0" xyz="0 0 0"/>
      <geometry>s
        <box size=".36 .4 .06"/>
      </geometry>
    </collision>
</link>"""

r_joint = """
  <joint name="theta" type="revolute">
    <parent link="base_link" />
    <child link="rod0" />
    <axis xyz="0 -1 0" />
    <origin xyz="0 0 0"/>
    <limit lower="0" upper="3.14159265359" />
  </joint>
"""

quadrotor_urdf = f"""<?xml version="1.0"?>
<robot name="quadrotor">
  {base_link}
  {rod_link}
  {r_joint}
</robot>"""

# quadrotor_urdf = f"""<?xml version="1.0"?>
# <robot name="quadrotor">
#   {base_link}
# </robot>"""

In [23]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    MultibodyPlant,
    Parser,
    Propeller,
    PropellerInfo,
    RigidTransform,
    BodyIndex
)
import pydrake.math
# from pydrake.math import SpatialInertia
from pydrake.examples import QuadrotorPlant

from underactuated.scenarios import AddFloatingRpyJoint


def MakeMultibodyQuadrotor():
    builder = DiagramBuilder()
    # The MultibodyPlant handles f=ma, but doesn't know about propellers.
    drone, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step = 0.0)
    parser = Parser(drone)
    ConfigureParser(parser)
    (model_instance, ) = parser.AddModelsFromString(quadrotor_urdf, "urdf")
    # scene_graph = builder.AddSystem(SceneGraph())
    print(builder)
    # AddMultiBodyPlantToBuilder(
    #      builder, drone.get_output_port(), scene_graph
    #  )
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty("/Background", "visible", False)
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    # By default the multibody has a quaternion floating base.  To match
    # QuadrotorPlant, we can manually add a FloatingRollPitchYaw joint. We set
    # `use_ball_rpy` to false because the BallRpyJoint uses angular velocities
    # instead of ṙ, ṗ, ẏ.
    AddFloatingRpyJoint(
        drone,
        drone.GetFrameByName("base_link"),
        model_instance,
        use_ball_rpy=False,
    )

    drone.Finalize()

    # Now we can add in propellers as an external force on the MultibodyPlant.
    body_index = drone.GetBodyByName("base_link").index()
    # Default parameters from quadrotor_plant.cc:
    L = 0.15  # Length of the arms (m).
    kF = 1.0  # Force input constant.
    kM = 0.0245  # Moment input constant.

    # Note: Rotors 0 and 2 rotate one way and rotors 1 and 3 rotate the other.
    prop_info = [
        PropellerInfo(body_index, RigidTransform([L, 0, 0]), kF, kM),
        PropellerInfo(body_index, RigidTransform([0, L, 0]), kF, -kM),
        PropellerInfo(body_index, RigidTransform([-L, 0, 0]), kF, kM),
        PropellerInfo(body_index, RigidTransform([0, -L, 0]), kF, -kM),
    ]
    propellers = builder.AddSystem(Propeller(prop_info))
    type(Propeller(prop_info))
    builder.Connect(
        propellers.get_output_port(),
        drone.get_applied_spatial_force_input_port(),
    )
    builder.Connect(
        drone.get_body_poses_output_port(),
        propellers.get_body_poses_input_port(),
    )
    builder.ExportInput(propellers.get_command_input_port(), "u")
    builder.ExportOutput(drone.get_state_output_port(),"outputport")

    return builder.Build(), drone, parser, builder




In [25]:
# Q = np.diag([10, 10, 10, 1, 1, 1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01])  # state cost matrix
Q = np.eye(14)
Q[:7, :7] = 10*Q[:7, :7] 
R = np.diag([1, 1, 1, 1])  # input cost matrix
TARGET_STATE = np.zeros(14)
MASS = 1.0
START_STATE = [0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0]

# Q = np.eye(12)
# Q[:6, :6] = 10*np.eye(6)
# R = np.diag([1, 1, 1, 1])  # input cost matrix
# MASS = 0.775
# TARGET_STATE = np.zeros(12)
# START_STATE = np.zeros(12)
# START_STATE[:3] = [1, 2, 3]

#def lqrtosys():
builder = DiagramBuilder()
diagram, drone,_, _ = MakeMultibodyQuadrotor()
qplantwpend = builder.AddSystem(diagram)
def lqrc(qplantwpend):
    diagram = qplantwpend
    diagram_context = diagram.CreateDefaultContext()
    g = 9.81 # m/s^2
    mass = MASS
    weight = mass*g
    diagram.get_input_port().FixValue(diagram_context,[weight/4,weight/4,weight/4,weight/4])
    diagram_context.SetContinuousState(TARGET_STATE)
    controller = LinearQuadraticRegulator(diagram, diagram_context, Q, R)

    return controller

tempvar = lqrc(qplantwpend)
controller = builder.AddSystem(tempvar)
builder.Connect(qplantwpend.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0),
            qplantwpend.get_input_port(0))

diagram = builder.Build()

#instantiate a simulator
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

# simulator.set_publish_every_time_step(True) # makes sim faster
context = simulator.get_mutable_context()
context.SetTime(0.0)
context.SetContinuousState(START_STATE)
simulator.Initialize()
simulator.AdvanceTo(4)

# simulator.get_mutable_integrator().set_fixed_step_mode(True)

# start recording the video for the animation of the simulation
# visualizer = diagram.GetSubsystemByName("visualizer")
# visualizer.StartRecording(False)

context=qplantwpend.CreateDefaultContext()
qplantwpend.ForcedPublish(context)




In [13]:
# diagram, drone,_, builder = MakeMultibodyQuadrotor()
# if diagram:
#     diagram_context = diagram.CreateDefaultContext()
#     g = 9.81 # m/s^2
#     mass = 1
#     weight = mass*g
#     Q = np.diag([10, 10, 10, 1, 1, 1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01])  # state cost matrix
#     R = np.diag([1, 1, 1, 1])  # input cost matrix
#     diagram.get_input_port().FixValue(diagram_context,[weight/4,weight/4,weight/4,weight/4])
#     diagram_context.SetContinuousState([0,0,0,0,0,0,np.pi,0,0,0,0,0,0,0])
#     controller = LinearQuadraticRegulator(diagram, diagram_context, Q, R)

#     # # breaks here
#     # builder.AddNamedSystem("lqr", controller)


#     # builder.Connect(drone.get_state_output_port(), controller.get_input_port(0))
#     # builder.Connect(controller.get_output_port(0),
#     #                 drone.get_actuation_input_port())


#     # meshcat.Delete()
#     # visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
#     # visualizer.set_name("visualizer")
# #return diagram


In [19]:
# diagram,drone,_ = MakeMultibodyQuadrotor()
# if diagram:

#     #builder = DiagramBuilder()

#     diagram_context = diagram.CreateDefaultContext()
#     # state_port = drone.get_state_output_port()
#     # context = drone.CreateDefaultContext()
#     # from above cell: base_link_mass = 0.775 & rod0_link_mass = 0.225
#     g = 9.81 # m/s^2
#     mass = 1
#     weight = mass*g
#     Q = np.diag([10, 10, 10, 1, 1, 1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01])  # state cost matrix
#     R = np.diag([1, 1, 1, 1])  # input cost matrix
#     diagram.get_input_port().FixValue(diagram_context,[weight/4,weight/4,weight/4,weight/4])
#     #diagram_context.SetContinuousState([0,0,0,0,0,0,(np.pi)/2,0,0,0,0,0,0,0]) # why doesnt it work w pi/2?
#     diagram_context.SetContinuousState([0,0,0,0,0,0,0,0,0,0,0,0,0,0])

#     #x0 = diagram_context.SetContinuousState([0,0,0,0,0,0,0,0,0,0,0,0,0,0])


#     # state_port = drone.get_state_output_port()
#     # context = drone.CreateDefaultContext()
#     # state = context.get_mutable_continuous_state_vector()
#     # state.SetFromVector([0,0,0,0,0,0,0,0,0,0,0,0,0,0])


#     controller = LinearQuadraticRegulator(diagram, diagram_context, Q, R)

#     # 4. synthesize a LinearQuadraticRegulator, and add it to the builder
#     #controller = LinearQuadraticRegulator(cartpole, cartpole_context, Q, R, input_port_index = cartpole.get_actuation_input_port().get_index())
#     builder.AddNamedSystem("lqr", controller)
#     # 5. wire cart-pole and lqr
#     builder.Connect(drone.get_state_output_port(), controller.get_input_port(0))
#     builder.Connect(controller.get_output_port(0),
#                     drone.get_actuation_input_port())


#     # add a visualizer
#     meshcat.Delete()
#     visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
#     visualizer.set_name("visualizer")

#     # finish building the block diagram
#     #diagram = builder.Build()
# return diagram


#     # We'll print the diagram context here so you can see the results of your
#     # code.
#     #print(diagram_context)
# #drone_diagram.ForcedPublish(context)


RuntimeError: DiagramBuilder: Build() or BuildInto() has already been called to create a Diagram; this DiagramBuilder may no longer be used.

In [ ]:
x_star = diagram_context.SetContinuousState([0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0])
Q = np.diag([10, 10, 10, 1, 1, 1, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01])  # state cost matrix
R = np.diag([1, 1, 1, 1])  # input cost matrix

def cartpole_balancing(quadrotor_urdf, x_star, Q, R):

    """
    Arguments: 
        cartpole_urdf: str
            a string that contains a urdf description of the system
        x_star: array
            a fixed point of the system; the desired equilibrium point 
            we want the system to reach
        Q: state cost matrix in LQR
        R: input cost matrix in LQR

    simulates and generates an animation for the lqr controlled system
    """

    # # builder = DiagramBuilder()

    # # cartpole, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    # # Parser(cartpole).AddModelsFromString(cartpole_urdf, "urdf")
    # # cartpole.Finalize()
    # # cartpole.set_name("cartpole")
    
    # # 1. obtain the context from the cartpole plant
    # #cartpole_context = cartpole.CreateDefaultContext()
    # #diagram_context = diagram.CreateDefaultContext() #already done in prev cell

    # #cartpole_context = cartpole.GetMyMutableContextFromRoot()
    # diagram_context = diagram.GetMyMutableContextFromRoot()



    # # 2. set the state vector in the context to x_star
    # #cartpole_context.get_mutable_continuous_state_vector().SetFromVector(x_star)
    # diagram_context.get_mutable_continuous_state_vector().SetFromVector(x_star)



    # # 3. fix the actuation input port to zero
    # #cartpole.get_actuation_input_port().FixValue(cartpole_context, [0])
    # diagram.get_input_port().FixValue(diagram_context,[weight/4,weight/4,weight/4,weight/4]) #not sure about this


    # # 4. synthesize a LinearQuadraticRegulator, and add it to the builder
    # controller = LinearQuadraticRegulator(cartpole, cartpole_context, Q, R, input_port_index = cartpole.get_actuation_input_port().get_index())
    # builder.AddNamedSystem("lqr", controller)
    # # 5. wire cart-pole and lqr
    # builder.Connect(cartpole.get_state_output_port(), controller.get_input_port(0))
    # builder.Connect(controller.get_output_port(0),
    #                 cartpole.get_actuation_input_port())


    # # add a visualizer
    # meshcat.Delete()
    # visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    # visualizer.set_name("visualizer")

    # # finish building the block diagram
    # diagram = builder.Build()
    # return diagram

In [ ]:
# def simulate_and_animate(diagram, x0, sim_time=5., visualize=True):
#     # instantiate a simulator
#     simulator = Simulator(diagram)
#     simulator.set_publish_every_time_step(False) # makes sim faster
#     simulator.get_mutable_integrator().set_fixed_step_mode(True)

#     # start recording the video for the animation of the simulation
#     visualizer = diagram.GetSubsystemByName("visualizer")
#     visualizer.StartRecording(False)

#     if (len(x0) !=
#             diagram.GetSubsystemByName("cartpole").num_continuous_states()):
#         print(f"Your plant doesn't have {len(x0)} state variables.")
#         return

#     # reset initial time and state
#     context = simulator.get_mutable_context()
#     context.SetTime(0.)
#     context.SetContinuousState(x0)

#     # run sim
#     simulator.Initialize()
#     simulator.AdvanceTo(sim_time)

#     # stop video
#     visualizer.PublishRecording()
#     visualizer.DeleteRecording()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=aa48ed2a-5654-48c0-baae-a6a8aa9d4792' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>